In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path)

## Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [3]:
browser.visit('https://mars.nasa.gov/news/')
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [4]:
bsoup = BeautifulSoup(browser.html, 'html.parser')
slide_elem = bsoup.select_one('ul.item_list li.slide')

In [5]:
slide_elem.find("div", class_='content_title')

<div class="content_title"><a href="/news/8744/nasa-engineers-checking-insights-weather-sensors/" target="_self">NASA Engineers Checking InSight's Weather Sensors</a></div>

In [6]:
news_title = slide_elem.find("div", class_='content_title').get_text()
print(f'"news_title" div contains : [{news_title}]')

"news_title" div contains : [NASA Engineers Checking InSight's Weather Sensors]


In [7]:
article_teaser = slide_elem.find('div', class_="article_teaser_body").get_text()
print(f'"article_teaser_body" div contains : [{article_teaser}]')

"article_teaser_body" div contains : [An electronics issue is suspected to be preventing the sensors from sharing their data about Mars weather with the spacecraft.]


## JPL Mars Space Images - Featured Image

In [8]:
jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_url)

In [9]:
image_elem = browser.find_by_id('full_image')
image_elem.click()

In [10]:
browser.is_element_present_by_text('more info', wait_time=1)
more_info_elem = browser.find_link_by_partial_text('more info')
more_info_elem.click()

In [11]:
img_pull = BeautifulSoup(browser.html, 'html.parser')
img_url_rel = img_pull.select_one('figure.lede a img').get("src")
img_url_rel

'/spaceimages/images/largesize/PIA01322_hires.jpg'

In [12]:
featured_image_url = f'https://www.jpl.nasa.gov{img_url_rel}'
print(f'featured_image_url : {featured_image_url}')

featured_image_url : https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA01322_hires.jpg


## Visit the Mars Weather twitter account here and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called mars_weather

In [13]:
browser.visit('https://twitter.com/marswxreport?lang=en')

time.sleep(4)

In [14]:
html = browser.html
weather_soup = BeautifulSoup(html, 'html.parser')

In [15]:
mars_weather_tweet = weather_soup.find('div', attrs={"class": "tweet", "data-name": "Mars Weather"})

In [16]:
try:
    mars_weather = mars_weather_tweet.find("p", "tweet-text").get_text()

except AttributeError:

    pattern = re.compile(r'sol')
    mars_weather = weather_soup.find('span', text=pattern).text
    
print(f'mars_weather : [{mars_weather}]')



mars_weather : [InSight sol 611 (2020-08-15) low -93.8ºC (-136.9ºF) high -15.9ºC (3.4ºF)
winds from the WNW at 7.3 m/s (16.3 mph) gusting to 17.9 m/s (40.2 mph)
pressure at 7.90 hPa]


## Visit the USGS Astrogeology site here to obtain high resolution images for each of Mar's hemispheres.

In [17]:
browser.visit('https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars')

In [18]:
hemisphere_images = []

all_prods_list = browser.find_by_css("a.product-item h3")
for i in range(len(all_prods_list)):
    mars_hemi_dict = {}
    browser.find_by_css("a.product-item h3")[i].click()

    sample_elem = browser.find_link_by_text('Sample').first
    mars_hemi_dict['img_url'] = sample_elem['href']
    mars_hemi_dict['title'] = browser.find_by_css("h2.title").text
    hemisphere_images.append(mars_hemi_dict)    
    browser.back()

In [19]:
 print(f'hemisphere_images : {"--"*20}\n\n {hemisphere_images}')

hemisphere_images : ----------------------------------------

 [{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg', 'title': 'Cerberus Hemisphere Enhanced'}, {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg', 'title': 'Schiaparelli Hemisphere Enhanced'}, {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg', 'title': 'Syrtis Major Hemisphere Enhanced'}, {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg', 'title': 'Valles Marineris Hemisphere Enhanced'}]


## Space Facts : Mars

In [20]:
space_facts_df = pd.read_html('https://space-facts.com/mars/')[0]
space_facts_df.columns=['description', 'value']
space_facts_df.set_index('description', inplace=True)
space_facts_df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [21]:
space_facts_df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

In [22]:
browser.quit()